<a href="https://colab.research.google.com/github/Kridr/HW_OSDA_HSE/blob/main/HW5_OSDA/HW5_OSDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np

# Task 1

Original data

In [2]:
df_orig = pd.read_csv('data.txt', sep='\t', index_col='No')
df_orig

,System,mount,price,con,snow,ice,dur,Accegrade
No,,,,,,,,
1,SK,F,206,1.9,1.4,1.8,2.7,F
2,SRK,F or R,520,2.1,0.8,3.8,2.3,F
3,SK,F,160,1.7,1.9,1.6,3.7,F
4,SK,F,213,1.7,2.0,2.4,3.4,F
5,SMS,F or R,598,1.6,2.4,2.7,2.8,F
6,SK,F,109,2.0,1.9,2.4,3.7,F
7,SRK,F or R,325,2.0,2.1,3.2,2.8,F
8,SMS,F or R,498,1.5,3.3,3.5,2.0,T
9,SRK,F or R,396,2.8,2.1,3.1,2.5,T


Function for creating mask based on two objects using different interval methods


Function for checking if some object match a mask

In [3]:
def create_mask(elem1, elem2, num_attr_strat):
    #assuming elem1 and elem2 have the same attributes with the same order
    mask = []
    for i in range(len(elem1)):
        if type(elem1[i]) in [np.int64, np.float64]:
            if num_attr_strat == 'interval':
                mask.append((min(elem1[i], elem2[i]), max(elem1[i], elem2[i])))
            elif num_attr_strat == 'min_inf':
                mask.append((min(elem1[i], elem2[i]), np.inf))
            elif num_attr_strat == 'max_inf':
                mask.append((max(elem1[i], elem2[i]), np.inf))
        elif elem1[i] == elem2[i]:
            mask.append(elem1[i])
        else:
            mask.append('*')

    return mask

def validate_mask(elem, mask):
    answer = True
    for i in range(len(mask)):
        if mask[i] != '*':
            if type(mask[i]) == tuple:
                answer = answer and (elem[i] >= mask[i][0] and elem[i] <= mask[i][1])
            else:
                answer = answer and mask[i] == elem[i]
        if not answer:
            break
    return answer

Function for performing classification

In [4]:
def fit_predict(df_train_X, df_train_y, df_test_X, num_attr_strat):
    total_preds = {}
    for i in df_test_X.index:
        print(f'Predicting element with index {i}: {df_test_X.loc[i].to_numpy()}')
        print(f'We need to create mask using 1st element of train set: {df_train_X.loc[df_train_X.index[0]].to_numpy()}')
        mask = create_mask(df_test_X.loc[i], df_train_X.loc[df_train_X.index[0]], num_attr_strat)
        print(f'Mask created with {i} element of test and 1st element from train: {mask}')
        predictions = [df_train_y.loc[df_train_y.index[0]]]

        for j in df_train_X.index:
            if j == df_train_X.index[0]:
                continue
            print(f'\t{j} element of train set: {df_train_X.loc[j].to_numpy()}')
            if validate_mask(df_train_X.loc[j], mask):
                print('\t\tThis element matches with created mask')
                predictions.append(df_train_y.loc[j])
                print(f'\t\tArray of predictions for the test example: {predictions}')
                
                all_same_preds = True
                for k in range(1, len(predictions)):
                    if predictions[k] != predictions[0]:
                        all_same_preds = False
                        break
                
                if not all_same_preds:
                    print(f'\t\tNot all predictions are the same, so the answer for element {i} is undefined')
                    total_preds[i] = None

            else:
                print('\t\tThis element does not match with created mask')
        
        if j == df_train_X.index[-1]:
            print(f'The answer for element with index {i} is {predictions[0]}')
            total_preds[i] = predictions[0]

        print('\n')

    print(f'Total predictions are {total_preds}')

    return pd.Series(total_preds)

Function for calculating of accuracy

In [5]:
def accuracy(df_test_y_true, df_test_y_pred):
    df_test_y_true_s = df_test_y_true.sort_index().to_numpy()
    df_test_y_pred_s = df_test_y_pred.sort_index().to_numpy()
    
    return (df_test_y_true_s == df_test_y_pred_s).sum() / len(df_test_y_true_s)

Function for creating train and test data

In [6]:
def create_X_y_train_test(df, test_ind, target):
    df_train = df[~df.index.isin(test_ind)]
    df_test = df[df.index.isin(test_ind)]

    df_train_X = df_train.drop(columns=[target])
    df_train_y = df_train[target]

    df_test_X = df_test.drop(columns=[target])
    df_test_y = df_test[target]

    return df_train_X, df_test_X, df_train_y, df_test_y

## 1. Consider the last three examples of this dataset only (no cross validation) and classify them using lazy classification with pattern structures (with interval pattern structures for numerical attributes). What is the accuracy of this classification? why do you think that happened?

Obtaining data

In [7]:
df_orig_train_X, df_orig_test_X, df_orig_train_y, df_orig_test_y = create_X_y_train_test(df_orig, df_orig.index[-3:], 'Accegrade')

Performing classification

In [8]:
df_orig_y_pred = fit_predict(df_orig_train_X, df_orig_train_y, df_orig_test_X, 'interval')

Predicting element with index 15: ['SK' 'F' 99 2.8 2.2 2.5 4.0]
We need to create mask using 1st element of train set: ['SK' 'F' 206 1.9 1.4 1.8 2.7]
Mask created with 15 element of test and 1st element from train: ['SK', 'F', (99, 206), (1.9, 2.8), (1.4, 2.2), (1.8, 2.5), (2.7, 4.0)]
	2 element of train set: ['SRK' 'F or R' 520 2.1 0.8 3.8 2.3]
		This element does not match with created mask
	3 element of train set: ['SK' 'F' 160 1.7 1.9 1.6 3.7]
		This element does not match with created mask
	4 element of train set: ['SK' 'F' 213 1.7 2.0 2.4 3.4]
		This element does not match with created mask
	5 element of train set: ['SMS' 'F or R' 598 1.6 2.4 2.7 2.8]
		This element does not match with created mask
	6 element of train set: ['SK' 'F' 109 2.0 1.9 2.4 3.7]
		This element matches with created mask
		Array of predictions for the test example: ['F', 'F']
	7 element of train set: ['SRK' 'F or R' 325 2.0 2.1 3.2 2.8]
		This element does not match with created mask
	8 element of train set

Predictions of the above classification

In [9]:
df_orig_y_pred

15    F
16    F
17    F
dtype: object

Accuracy of the above classification

In [10]:
accuracy(df_orig_test_y, df_orig_y_pred)

0.0

## 2. Now let’s consider the same dataset, with a different order below, Try to classify same three examples, what is the accuracy?

Reading data from file

In [11]:
df_mixed = pd.read_csv('data_mixed.txt', sep='\t', index_col='No')
df_mixed.head()

,System,mount,price,con,snow,ice,dur,Accegrade
No,,,,,,,,
1,SK,F,149,1.9,2.5,4.0,3.8,T
2,SRK,F or R,520,2.1,0.8,3.8,2.3,F
3,SRK,F or R,389,2.0,2.2,3.3,4.3,T
4,SK,F,213,1.7,2.0,2.4,3.4,F
5,SMS,F or R,598,1.6,2.4,2.7,2.8,F


Obtaining train and test data

In [12]:
df_mixed_train_X, df_mixed_test_X, df_mixed_train_y, df_mixed_test_y = create_X_y_train_test(df_mixed, df_mixed.index[-3:], 'Accegrade')

Performing classification of the above data

In [13]:
df_mixed_y_pred = fit_predict(df_mixed_train_X, df_mixed_train_y, df_mixed_test_X, 'interval')

Predicting element with index 15: ['SK' 'F' 99 2.8 2.2 2.5 4.0]
We need to create mask using 1st element of train set: ['SK' 'F' 149 1.9 2.5 4.0 3.8]
Mask created with 15 element of test and 1st element from train: ['SK', 'F', (99, 149), (1.9, 2.8), (2.2, 2.5), (2.5, 4.0), (3.8, 4.0)]
	2 element of train set: ['SRK' 'F or R' 520 2.1 0.8 3.8 2.3]
		This element does not match with created mask
	3 element of train set: ['SRK' 'F or R' 389 2.0 2.2 3.3 4.3]
		This element does not match with created mask
	4 element of train set: ['SK' 'F' 213 1.7 2.0 2.4 3.4]
		This element does not match with created mask
	5 element of train set: ['SMS' 'F or R' 598 1.6 2.4 2.7 2.8]
		This element does not match with created mask
	6 element of train set: ['SK' 'F' 109 2.0 1.9 2.4 3.7]
		This element does not match with created mask
	7 element of train set: ['SRK' 'F or R' 325 2.0 2.1 3.2 2.8]
		This element does not match with created mask
	8 element of train set: ['SMS' 'F or R' 498 1.5 3.3 3.5 2.0]
		Th

Results of the above classification

In [14]:
df_mixed_y_pred

15    T
16    T
17    T
dtype: object

Accuracy of the above classification

In [15]:
accuracy(df_mixed_test_y, df_mixed_y_pred)

1.0

## 3. Assess the performance of the lazy classification with interval pattern structures on the randomly ordered sample (the second one) by applying 6-cross validation

Function for performing cross-validation

In [16]:
def cross_validation(df, n, target, num_attr_strat):
    step = len(df.index) // n + 1
    test_columns_list = [list(df.index[i:max(i+step, min(df.index + 1))]) for i in range(0, len(df.index), step)]
    print(f'Test indexes for each fit_predict: {test_columns_list}')

    accuracies = []

    for test_columns in test_columns_list:
        print(f'Considering test columns: {test_columns}\n')
        df_train_X, df_test_X, df_train_y, df_test_y = create_X_y_train_test(df, test_columns, target)

        df_test_y_pred = fit_predict(df_train_X, df_train_y, df_test_X, num_attr_strat)

        cur_accuracy = accuracy(df_test_y, df_test_y_pred)
        print(f'Accuracy is {cur_accuracy}')
        accuracies.append(cur_accuracy)

        print('---------------------------------------------')

    print(f'All accuracies: {accuracies}')

    mean_accuracy = np.mean(accuracies)
    print(f'Mean accuracy: {mean_accuracy}')

    return mean_accuracy

Performing cross validation of the second variant of data

In [17]:
mean_accuracy = cross_validation(df_mixed, 6, 'Accegrade', 'interval')

Test indexes for each fit_predict: [[1, 2, 3], [4, 5, 6], [7, 8, 9], [10, 11, 12], [13, 14, 15], [16, 17]]
Considering test columns: [1, 2, 3]

Predicting element with index 1: ['SK' 'F' 149 1.9 2.5 4.0 3.8]
We need to create mask using 1st element of train set: ['SK' 'F' 213 1.7 2.0 2.4 3.4]
Mask created with 1 element of test and 1st element from train: ['SK', 'F', (149, 213), (1.7, 1.9), (2.0, 2.5), (2.4, 4.0), (3.4, 3.8)]
	5 element of train set: ['SMS' 'F or R' 598 1.6 2.4 2.7 2.8]
		This element does not match with created mask
	6 element of train set: ['SK' 'F' 109 2.0 1.9 2.4 3.7]
		This element does not match with created mask
	7 element of train set: ['SRK' 'F or R' 325 2.0 2.1 3.2 2.8]
		This element does not match with created mask
	8 element of train set: ['SMS' 'F or R' 498 1.5 3.3 3.5 2.0]
		This element does not match with created mask
	9 element of train set: ['SRK' 'F or R' 396 2.8 2.1 3.1 2.5]
		This element does not match with created mask
	10 element of train set: 

## 4. Classify the example number 15 using "min\_inf" pattern structures

Obtaining train and test data

In [18]:
df_mixed_inf_train_X, df_mixed_inf_test_X, df_mixed_inf_train_y, df_mixed_inf_test_y = create_X_y_train_test(df_mixed, [15], 'Accegrade')

Performing classification

In [19]:
df_mixed_min_inf_test_y_pred = fit_predict(df_mixed_inf_train_X, df_mixed_inf_train_y, df_mixed_inf_test_X, 'min_inf')

accuracy_mixed_min_inf = accuracy(df_mixed_inf_test_y, df_mixed_min_inf_test_y_pred)

Predicting element with index 15: ['SK' 'F' 99 2.8 2.2 2.5 4.0]
We need to create mask using 1st element of train set: ['SK' 'F' 149 1.9 2.5 4.0 3.8]
Mask created with 15 element of test and 1st element from train: ['SK', 'F', (99, inf), (1.9, inf), (2.2, inf), (2.5, inf), (3.8, inf)]
	2 element of train set: ['SRK' 'F or R' 520 2.1 0.8 3.8 2.3]
		This element does not match with created mask
	3 element of train set: ['SRK' 'F or R' 389 2.0 2.2 3.3 4.3]
		This element does not match with created mask
	4 element of train set: ['SK' 'F' 213 1.7 2.0 2.4 3.4]
		This element does not match with created mask
	5 element of train set: ['SMS' 'F or R' 598 1.6 2.4 2.7 2.8]
		This element does not match with created mask
	6 element of train set: ['SK' 'F' 109 2.0 1.9 2.4 3.7]
		This element does not match with created mask
	7 element of train set: ['SRK' 'F or R' 325 2.0 2.1 3.2 2.8]
		This element does not match with created mask
	8 element of train set: ['SMS' 'F or R' 498 1.5 3.3 3.5 2.0]
		Th

Accuracy of the above classification

In [20]:
print(f'Accuracy is {accuracy_mixed_min_inf}')

Accuracy is 1.0


## 5. Classify the example number 15 using "max\_inf" pattern structures

Performing classfication

In [21]:
df_mixed_max_inf_test_y_pred = fit_predict(df_mixed_inf_train_X, df_mixed_inf_train_y, df_mixed_inf_test_X, 'max_inf')

accuracy_mixed_max_inf = accuracy(df_mixed_inf_test_y, df_mixed_max_inf_test_y_pred)

Predicting element with index 15: ['SK' 'F' 99 2.8 2.2 2.5 4.0]
We need to create mask using 1st element of train set: ['SK' 'F' 149 1.9 2.5 4.0 3.8]
Mask created with 15 element of test and 1st element from train: ['SK', 'F', (149, inf), (2.8, inf), (2.5, inf), (4.0, inf), (4.0, inf)]
	2 element of train set: ['SRK' 'F or R' 520 2.1 0.8 3.8 2.3]
		This element does not match with created mask
	3 element of train set: ['SRK' 'F or R' 389 2.0 2.2 3.3 4.3]
		This element does not match with created mask
	4 element of train set: ['SK' 'F' 213 1.7 2.0 2.4 3.4]
		This element does not match with created mask
	5 element of train set: ['SMS' 'F or R' 598 1.6 2.4 2.7 2.8]
		This element does not match with created mask
	6 element of train set: ['SK' 'F' 109 2.0 1.9 2.4 3.7]
		This element does not match with created mask
	7 element of train set: ['SRK' 'F or R' 325 2.0 2.1 3.2 2.8]
		This element does not match with created mask
	8 element of train set: ['SMS' 'F or R' 498 1.5 3.3 3.5 2.0]
		T

Accuracy of the above classifcaiton

In [22]:
print(f'Accuracy is {accuracy_mixed_max_inf}')

Accuracy is 1.0


# Task 2

In [23]:
import numpy as np
from pprint import pprint

Defining positive and negative molecules

In [24]:
positives = [
    [('OH', 'CH3'), 'C', 'C', ('I', 'OH')],
    [('OH', 'OH'), 'C', 'C', ('NH2', 'CH3')],
    [('I', 'CH3'), 'C', 'C', ('CH3', 'NH2')],
    [('CH3', 'I'), 'C', 'C', ('I', 'NH2')],
]

negatives = [
    [('NH2', 'NH2'), 'C', 'C', ('I', 'OH')],
    [('I', 'NH2'), 'C', 'C', ('I', 'OH')],
    [('OH', 'OH'), 'C', 'C', ('NH2', 'I')],
]

Function that collects features from a molecule

In [25]:
def get_feats(mol):
    #Means X--C===C--Z
    #         | 
    #         Y   
    top_subbot1 = frozenset([tuple(sorted(mol[0])), mol[-1][0]])
    top_subbot2 = frozenset([tuple(sorted(mol[0])), mol[-1][1]])
    bot_subtop1 = frozenset([tuple(sorted(mol[-1])), mol[0][0]])
    bot_subtop2 = frozenset([tuple(sorted(mol[-1])), mol[0][1]])

    #Means X--C===C
    #         | 
    #         Y   
    top = frozenset(mol[0])
    bot = frozenset(mol[-1])

    #Means X---C===C---Y
    cross1  = frozenset([mol[0][0], mol[-1][0]])
    cross2  = frozenset([mol[0][0], mol[-1][1]])
    cross3  = frozenset([mol[0][1], mol[-1][0]])
    cross4  = frozenset([mol[0][1], mol[-1][1]])

    #Means C===C---X
    single1 = frozenset({mol[0][0]})
    single2 = frozenset({mol[0][1]})
    single3 = frozenset({mol[-1][0]})
    single4 = frozenset({mol[-1][1]})

    return {
        '3': {top_subbot1, top_subbot2, bot_subtop1, bot_subtop2},
        '2_os': {top, bot},
        '2_ts': {cross1, cross2, cross3, cross4},
        '1': {single1, single2, single3, single4}
    }

Obtaining features for each posiive and negative molecules

In [26]:
positives_feats = np.array([get_feats(positive) for positive in positives])
negatives_feats = np.array([get_feats(negative) for negative in negatives])

Function that applies diamond operator to a list of features

In [27]:
def diamond(feat_list):
    total_feat = feat_list[0].copy()
    for feat in feat_list[1:]:
        for key in total_feat.keys():
            total_feat[key] = total_feat[key].intersection(feat[key])

    return total_feat

Function that applies double diamond operator to a list of features

In [28]:
def double_diamond(feat_list, full_feat_list):
    total_feat = diamond(feat_list)

    match_feat_list = []
    for feat in full_feat_list:
        match = True
        for key in feat.keys():
            match = match and (total_feat[key] | feat[key] == feat[key])
            if not match:
                break
        if match:
            match_feat_list.append(feat)
    
    return match_feat_list

Function that returns all combinations of elements

In [29]:
import itertools

def get_combs(A, full=True):
    """
    A: list from which we need to find combinations
    full: flag; if full=True then A itself is included in the output combinations
    """
    combs = []
    finish = len(A) + 1 if full else len(A)
    for i in range(2, finish):
        combs += list(itertools.combinations(A, i))

    combs = [list(x) for x in combs]

    return combs

Function for finding concepts (size >= 2)

In [30]:
def find_concepts(full_feat_list, names_si):
    index_list = list(range(len(full_feat_list)))
    index_combs = get_combs(index_list)
    print(f'All combinations of elements: {index_combs}\n')
    results = []
    for index_comb in index_combs:
        names = ['G' + str(i + names_si) for i in index_comb]
        print(f'Considering {names} elements')
        feat_list = list(full_feat_list[index_comb])
        print('\tFeatures of molecules:')
        pprint(feat_list)
        # for feat in feat_list:
        #     pprint(f'\t\t{feat}')

        d_res = diamond(feat_list)
        print('\n\tResults of diamond operator')
        pprint(d_res)
        dd_res = double_diamond(feat_list, full_feat_list)
        print('\n\tResults of double diamond operator')
        pprint(dd_res)

        if (feat_list == dd_res):
            print('\n\tThis is a concept!')
            results.append((names, d_res))
        else:
            print('\n\tThis is not a concept (see results of double diamond operator)')

        print('---------------------------------------------\n\n')

    return results

## Finding concepts

Finding concepts of positive data

In [31]:
concepts_pos = find_concepts(positives_feats, 1)

All combinations of elements: [[0, 1], [0, 2], [0, 3], [1, 2], [1, 3], [2, 3], [0, 1, 2], [0, 1, 3], [0, 2, 3], [1, 2, 3], [0, 1, 2, 3]]

Considering ['G1', 'G2'] elements
	Features of molecules:
[{'1': {frozenset({'CH3'}), frozenset({'OH'}), frozenset({'I'})},
  '2_os': {frozenset({'I', 'OH'}), frozenset({'CH3', 'OH'})},
  '2_ts': {frozenset({'I', 'OH'}),
           frozenset({'CH3', 'I'}),
           frozenset({'OH'}),
           frozenset({'CH3', 'OH'})},
  '3': {frozenset({('CH3', 'OH'), 'I'}),
        frozenset({('I', 'OH'), 'CH3'}),
        frozenset({('I', 'OH'), 'OH'}),
        frozenset({('CH3', 'OH'), 'OH'})}},
 {'1': {frozenset({'CH3'}), frozenset({'NH2'}), frozenset({'OH'})},
  '2_os': {frozenset({'NH2', 'CH3'}), frozenset({'OH'})},
  '2_ts': {frozenset({'CH3', 'OH'}), frozenset({'NH2', 'OH'})},
  '3': {frozenset({'OH', ('CH3', 'NH2')}),
        frozenset({('OH', 'OH'), 'CH3'}),
        frozenset({'NH2', ('OH', 'OH')})}}]

	Results of diamond operator
{'1': {frozenset({'CH3

In [32]:
concepts_pos

[(['G1', 'G2'],
  {'3': set(),
   '2_os': set(),
   '2_ts': {frozenset({'CH3', 'OH'})},
   '1': {frozenset({'CH3'}), frozenset({'OH'})}}),
 (['G2', 'G3'],
  {'3': set(),
   '2_os': {frozenset({'CH3', 'NH2'})},
   '2_ts': set(),
   '1': {frozenset({'CH3'}), frozenset({'NH2'})}}),
 (['G3', 'G4'],
  {'3': {frozenset({('CH3', 'I'), 'NH2'})},
   '2_os': {frozenset({'CH3', 'I'})},
   '2_ts': {frozenset({'CH3', 'I'}),
    frozenset({'CH3', 'NH2'}),
    frozenset({'I', 'NH2'})},
   '1': {frozenset({'CH3'}), frozenset({'NH2'}), frozenset({'I'})}}),
 (['G1', 'G3', 'G4'],
  {'3': set(),
   '2_os': set(),
   '2_ts': {frozenset({'CH3', 'I'})},
   '1': {frozenset({'CH3'}), frozenset({'I'})}}),
 (['G2', 'G3', 'G4'],
  {'3': set(),
   '2_os': set(),
   '2_ts': set(),
   '1': {frozenset({'CH3'}), frozenset({'NH2'})}}),
 (['G1', 'G2', 'G3', 'G4'],
  {'3': set(), '2_os': set(), '2_ts': set(), '1': {frozenset({'CH3'})}})]

Finding concepts of negative data

In [33]:
concepts_neg = find_concepts(negatives_feats, 5)

All combinations of elements: [[0, 1], [0, 2], [1, 2], [0, 1, 2]]

Considering ['G5', 'G6'] elements
	Features of molecules:
[{'1': {frozenset({'NH2'}), frozenset({'OH'}), frozenset({'I'})},
  '2_os': {frozenset({'I', 'OH'}), frozenset({'NH2'})},
  '2_ts': {frozenset({'NH2', 'OH'}), frozenset({'NH2', 'I'})},
  '3': {frozenset({('NH2', 'NH2'), 'OH'}),
        frozenset({'NH2', ('I', 'OH')}),
        frozenset({('NH2', 'NH2'), 'I'})}},
 {'1': {frozenset({'NH2'}), frozenset({'OH'}), frozenset({'I'})},
  '2_os': {frozenset({'I', 'OH'}), frozenset({'NH2', 'I'})},
  '2_ts': {frozenset({'I', 'OH'}),
           frozenset({'NH2', 'I'}),
           frozenset({'NH2', 'OH'}),
           frozenset({'I'})},
  '3': {frozenset({'I', ('I', 'NH2')}),
        frozenset({('I', 'OH'), 'I'}),
        frozenset({'NH2', ('I', 'OH')}),
        frozenset({'OH', ('I', 'NH2')})}}]

	Results of diamond operator
{'1': {frozenset({'NH2'}), frozenset({'OH'}), frozenset({'I'})},
 '2_os': {frozenset({'I', 'OH'})},
 '2_

In [34]:
concepts_neg

[(['G5', 'G6'],
  {'3': {frozenset({('I', 'OH'), 'NH2'})},
   '2_os': {frozenset({'I', 'OH'})},
   '2_ts': {frozenset({'NH2', 'OH'}), frozenset({'I', 'NH2'})},
   '1': {frozenset({'NH2'}), frozenset({'OH'}), frozenset({'I'})}}),
 (['G6', 'G7'],
  {'3': {frozenset({('I', 'NH2'), 'OH'})},
   '2_os': {frozenset({'I', 'NH2'})},
   '2_ts': {frozenset({'I', 'OH'}), frozenset({'NH2', 'OH'})},
   '1': {frozenset({'NH2'}), frozenset({'OH'}), frozenset({'I'})}}),
 (['G5', 'G6', 'G7'],
  {'3': set(),
   '2_os': set(),
   '2_ts': {frozenset({'NH2', 'OH'})},
   '1': {frozenset({'NH2'}), frozenset({'OH'}), frozenset({'I'})}})]

Function for checking if a result of diamond operation (second elements of above pairs) is contained in some of the element.

It is just a support function: minimal hypotheses are chosen manually, based on the function results

## Minimal hypotheses

Function for checking if at least on object matches single result of diamond function

In [35]:
def is_in(diamond_res, feats):
    single_cond = True
    for key in feats.keys():
        single_cond = single_cond and (diamond_res[key] | feats[key] == feats[key])

    return single_cond

def is_subset(diamond_res, feats_list):
    for feats in feats_list:
        single_cond = is_in(diamond_res, feats)

        if single_cond:
            return True
    return False

Function for checking if at least on object matches results of diamond function

In [36]:
def is_subsets(concepts, other_feats):
    for concept in concepts:
        print(f'Considering {concept[0]} concept')
        if is_subset(concept[1], other_feats):
            print('\tResult of diamond operator matches object(s) from feat list')
        else:
            print('\tResult of diamond operator does not match object(s) from feat list')

Applying above function for diamond results of positive concepts and to negative molecules

In [37]:
is_subsets(concepts_pos, negatives_feats)

Considering ['G1', 'G2'] concept
	Result of diamond operator does not match object(s) from feat list
Considering ['G2', 'G3'] concept
	Result of diamond operator does not match object(s) from feat list
Considering ['G3', 'G4'] concept
	Result of diamond operator does not match object(s) from feat list
Considering ['G1', 'G3', 'G4'] concept
	Result of diamond operator does not match object(s) from feat list
Considering ['G2', 'G3', 'G4'] concept
	Result of diamond operator does not match object(s) from feat list
Considering ['G1', 'G2', 'G3', 'G4'] concept
	Result of diamond operator does not match object(s) from feat list


Applying above function for diamond results of negative concepts and to positive molecules

In [38]:
is_subsets(concepts_neg, positives_feats)

Considering ['G5', 'G6'] concept
	Result of diamond operator does not match object(s) from feat list
Considering ['G6', 'G7'] concept
	Result of diamond operator does not match object(s) from feat list
Considering ['G5', 'G6', 'G7'] concept
	Result of diamond operator does not match object(s) from feat list


As a result:

Minimal positive hypotheses is diamond of ['G1', 'G2', 'G3', 'G4'] concept

Minimal negative hypotheses is diamond of ['G5', 'G6', 'G7'] concept

## Classification

In [39]:
tests = [
    [('CH3', 'CH3'), 'C', 'C', ('I', 'NH2')],
    [('NH2', 'I'), 'C', 'C', ('I', 'OH')],
    [('CH3', 'NH2'), 'C', 'C', ('I', 'OH')],
    [('CH3', 'NH2'), 'C', 'C', ('NH2', 'CH3')]
]

In [40]:
tests_feats = np.array([get_feats(test) for test in tests])

Function for classification using mininal hypotheses and test molecults.

Actually, we need another function for merging of a hypothesis features (if several concepts are in hypothesis). But as we just have one concept per each hypothesis, we don't need it

In [41]:
def classificate(min_pos_hyp, min_neg_hyp, tests_feats, name_offset):
    res = []
    for i, test_feats in enumerate(tests_feats):
        print(f'Object G{i+name_offset}')
        is_pos = is_in(min_pos_hyp, test_feats)
        is_neg = is_in(min_neg_hyp, test_feats)

        if is_pos and is_neg or (not is_pos and not is_neg):
            print('\tThis object classified indefinitely')
            res.append(None)
        elif is_pos:
            print('\tThis object classified positively')
            res.append(True)
        else:
            print('\tThis object classified negatively')
            res.append(False)
    return res

Results of classification

In [42]:
classificate(concepts_pos[-1][1], concepts_neg[-1][1], tests_feats, 8)

Object G8
	This object classified positively
Object G9
	This object classified negatively
Object G10
	This object classified indefinitely
Object G11
	This object classified positively


[True, False, None, True]